# 1. Flexible type-decarations
Type-declarations consists of a number of clauses, some of which are used only in, for example, act- or duty-type declarations, whereas others can be used in all type-declarations. In the first versions of eFLINT, and depending on which kind of type is declared, certain clauses are mandatory and need to be written in a fixed order. In eflint-3.0, a lot more flexibility is introduced, making additional clauses optional and allowing many clauses to be written in any order. This notebook clarifies the exact rules applicable to the different kinds of type-declarations.

Since eflint-2.0, there are two types of type-declarations: declarations that *introduce* a new type (or replace an existing one) and declarations that *extend* an existing type. The clauses that can be written in a type extension are henceforth refered to as the *accumulating* clauses, the other clauses are *domain-related* clauses. The sections of this notebook discuss the domain-related and accumulating clauses of the different kinds of type-declarations.

## 1.2 Fact-type declarations


#### domain-related clauses

The domain-related clauses establish the *domain* from which the values are taken that 'populate' the declared type, such as the `Identified by ...` clause of fact-type declarations. This clause is optional, with the default `Identified by String` being implicitly inserted when omitted. Thus the following declarations are identical. 

In [1]:
Fact user 
Fact user Identified by String

#### accumulating clauses
The accumulating clauses of a type-declaration can be written in any order. In fact, multiple occurrences of the same accumulating clause can appear in a single type-declarations. Internally, this is realised by considering such clauses as extensions of the type being declared. The following declarations of `admin` are therefore identifical. Accumulating clauses are always written behind domain-related clauses.

In [2]:
Fact logged-in Identified by user
Fact access-rights-of Identified by user * int
Fact admin Identified by user
  Conditioned by logged-in(user) // can also be comma-separated
  Conditioned by access-rights-of(user, 1)

In [3]:
Fact admin Identified by user
Extend Fact admin
  Conditioned by logged-in(user)
Extend Fact admin
  Conditioned by access-rights-of(user, 1)

Accumulating clauses are called accumulating because multiple of these can be written, whether in a single declaration or across various declarations, and their effects accumulate.

The accumulating clauses of a fact-type declaration are:

* Conditioned by
* Holds when
* Derived from

All conditions specified using `Conditioned by` clauses must hold true for an instance of the specified type to be enabled. If (at least) one derivation rule (`Holds when` or `Derived from`) derives an instance of a type then the instance holds true if all its conditions hold true.

## 1.3 Event-type declarations
#### domain-related clauses
Instead of `Identified by` even-type declarations use `Related to` followed by a list of comma-separated types specifying the formal parameters of the type, bound in the type's clauses. The `Related to` clauses is optional. When omitted, the defined type has no parameters and has only one instance, identified by the name of the type.
#### accumulating clauses
The accumulating clauses of an event-type are:
* Conditioned by
* Holds when
* Derived from
* Creates
* Terminates
* Syncs with

The effects of all post-conditions (`Creates` and `Terminates` clauses) manifest when an action/event is triggered and all instances computed from `Syncs with` clauses demand synchronisation when an action/event is triggered.

## 1.4 Act-type declarations
#### domain-related clauses
An act-type declaration associates a performing actor and an optional recipient actor with the type using the `Actor` and `Recipient` clauses respectively. Both can be ommitted. If `Actor` is ommitted, it is implicitly present as `Actor actor`, with `actor` a builtin type with values taken from the domain of strings. If `Recipient` is ommitted, then only one actor is associated with the type, namely the performing actor.

The one or two actors of an act-type are the first formal parameters of the type. Additional formal parameters can be specified with a `Related to` clause. The domain-related clauses are to be written in the order they are mentioned here.
#### accumulating clauses
The accumulating clauses of an act-type are:
* Conditioned by
* Holds when
* Derived from
* Creates
* Terminates
* Syncs with

The effects of all post-conditions (`Creates` and `Terminates` clauses) manifest when an action/event is triggered and all instances computed from `Syncs with` clauses demand synchronisation when an action/event is triggered.

## 1.5 Duty-type declarations
#### domain-related clauses
A duty-type declaration associates a duty-holding actor and a claimant actor with the type using the `Holder` and `Claimant` clauses respectively. Neither can be ommitted. 

The two actors of a duty-type are the first formal parameters of the type. Additional formal parameters can be specified with a `Related to` clause. The domain-related clauses are to be written in the order they are mentioned here.

#### accumulating clauses
The accumulating clauses of a duty-type are:
* Conditioned by
* Holds when
* Derived from
* Violated when
* Enforced by

If any of the violation conditions holds true while the duty holds true, the duty is considered violated. Similarly, if any of the actions computed from the expressions written in the `Enforced by` clauses is enabled while the duty holds true, the duty is considered violated.

## 1.6 Domain constraints
Immediately following the domain-related clauses of a type-declaration, an optional *domain constraint* can be written. For example, the domain constraint `Where ...` in the example below ensures that one cannot be once's own parent, i.e. that for all `A`, `parent-of(A,A)` is not a valid instance of the type `parent-of`:

In [4]:
Fact person
Fact parent-of Identified by person1 * person2 Where person1 != person2

The effects of domain-constraints are mostly noticeable when the instances of a type with a domain constraint are enumerable. For example, when executable actions are listed (e.g. in the REPL) or when derivation rules are evaluated. The following extension of `parent-of` makes it that every pair of two different persons are considered each other's parent.

In [5]:
+person(Alice). +person(Bob). +person(Chloe).
Extend Fact parent-of Derived from parent-of(person1, person2)

+person("Alice")
+person("Bob")
+person("Chloe")
+parent-of(person("Alice"),person("Bob"))
+parent-of(person("Alice"),person("Chloe"))
+parent-of(person("Bob"),person("Alice"))
+parent-of(person("Bob"),person("Chloe"))
+parent-of(person("Chloe"),person("Alice"))
+parent-of(person("Chloe"),person("Bob"))


## 1.7 Open and Closed type modifiers
Type declarations can be preceded by either the `Closed` or `Open` modifiers, indicating whether the closed-world assumption applies to the type or not (see Section 5). When ommitted, the `Closed` modifier is applied by default.